# Data Preperation

In [72]:
import pandas as pd
import numpy as np
import glob
import copy
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [73]:
countries={ 'Austria': 'AT', 'Belgium': 'BE',  'Bulgaria': 'BG', 'Switzerland': 'CH', 'Czech Republic': 'CZ', 'Germany': 'DE', 'Denmark': 'DK', 'Estonia': 'EE', 'Spain': 'ES', 'Finland': 'FI', 'France': 'FR',  'Greece': 'GR', 'Hungary': 'HU', 'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Latvia': 'LV', 'Montenegro': 'ME','Netherlands': 'NL', 'Norway': 'NO', 'Poland': 'PL', 'Portugal': 'PT', 'Serbia': 'RS', 'Sweden': 'SE', 'Slovenia': 'SI', 'Slovakia': 'SK', 'United Kingdom': 'UK'}

abbr_list=list(countries.values())

## Making the first,second,third and fourth columns of the dataframe as date,month,year and time

In [74]:
data=pd.DataFrame()
temp=pd.read_csv('../Data Sources/ENTSO-E/2018/Load/Croatia.csv')
data['Day']=temp['Time (CET)'].str[:2]
data['Month']=temp['Time (CET)'].str[3:5]
data['Year']=temp['Time (CET)'].str[6:10]
data['Time']=temp['Time (CET)'].str[11:16]+' - '+temp['Time (CET)'].str[29:35]
data = data.drop(range(1994, 1995)).reset_index(drop=True)

##

In [75]:
# In the country speciic Load data set, Austria,Belgium,Germany,Hungary,Netherlands report data every 15 minutes. 
# Therefore, these countries have 35044 data points per year. 
# UK and Ireland report data every 30 minutes henece these countries have 17522 datapoints per year. 
# All the others report every 1 hour hence have 8761 datapoints per year. 
# In Genearion dataset, situation is same as abobe except Belgium reports hourly data hence have 8761 datapoints. 
# In Transmission dataset, all countries report data hourly except Germany which reports every 15 minutes. 
# Therefroe,it is easy if all the data are converted to hourly data. 
# To do that in the countries with 35044 datapoints, mean is calculated in every successive 4 datapoints. 
# In the countries with 17522 datapoints, mean is calculated in every successive 2 datapoints.`

# Due to day light saving, all the datasets have null values on 25th March from 02:00 - 03:00.
# Considering the time intervals each country update the data, a total number of rows of 4,2 or 1 are dropped from the 31st March, 02:00 - 03:00 time interval.
# Also this returns a integer ('divider') based on the file length to get the energy values in a later step. In 15 min interval files this is 4, in 30 min interval files this is 2 and in 1 hour interval files this is 1.

def hourly_data(df):
    length=len(df.index)
    if length==35044:
        df = df.drop(range(7976, 7980)).reset_index(drop=True)
        divider=4
    elif length==17522:
        df = df.drop(range(3988, 3990)).reset_index(drop=True)
        divider=2
    else:
        df = df.drop(range(1994, 1995)).reset_index(drop=True)
        divider=1

    # Following command creates a numpy array with a length similar to the length of the dataframe. 
    # Values of the array are obtained by getting the floor division of the length value. 
    # For example, when divider=4, this array will be [0,0,0,0,1,1,1,1,2,2,2,2,....]. 
    # Then the rows of the dataframe will be grouped according to the order of the numpy array with the mean value of those 4 rows. 
    # For example, in the numpy array first 4 values are similar. Accordingly first 4 rows of the dataframe will be grouped and get the mean value of those rows 
    
    df=df.groupby(np.arange(len(df))//divider).mean()
    return(df)    


# 1. Preparing Load Data

In [76]:
def load(countries):

    load_dic = {}
    load_data=pd.DataFrame()
    load_data[['Day','Month','Year','Time']]=data[['Day','Month','Year','Time']]

    # In the following command we open the csv file of each country and save the data in the 'temp' dataframe. 
    # Then we call the 'hourly_data' function to make all the time steps to hourly data. 
    # Then the column 'Actual Total Load [MW] - {country_name} ({country_code})' is saved in the new dataframe 'load_data' under the column name '{country_code}'. 
    # For example, the column 'Actual Total Load [MW] - Germany (DE)' in the 'temp' dataframe will be saved in the 'load_data' dataframe under the column name 'DE'.
    
    for country,abbr in countries.items():
        temp=pd.read_csv(f'../Data Sources/ENTSO-E/2018/Load/{country}.csv')
        # temp = temp.replace(['n/e',np.nan] ,0)
        temp = temp.replace(['n/e'], 0)
        temp = hourly_data(temp)
        load_data[f'{abbr}']=temp[f'Actual Total Load [MW] - {country} ({abbr})']
        temp['demand'] = temp[f'Actual Total Load [MW] - {country} ({abbr})']
        load_dic[abbr] = temp[['demand']]

    return(load_data,load_dic)


# 2. Preparing Generation Data

In [77]:
def generation(countries):
    
    generation_dic = {}
    generation_data=pd.DataFrame()
    generation_data[['Day','Month','Year','Time']]=data[['Day','Month','Year','Time']]

    # In the following command we open the csv file of each country and save the data in the 'temp' dataframe. 
    # Then we make all the 'n/e' values of the 'temp' 0.
    # Then we copy the hydro pumped storage consumption data to load_data of the respective country
    # then we remove the 'Hydro Pumped Storage  - Actual Consumption [MW]' column
    # Then we call the 'hourly_data' function to make all the time steps to hourly data. 
    # Then we remove the columns in which a single data is not recorded
    # Then we get the column names of the 'temp' dataframe into a numpy array called 'fuels' and get the column name without the '- Actual Aggregated [MW]' part.
    # Then we change the column names of the dataframe with the edited names in the previous step.
    # Then we update the 'generation_data' dataframe and 'generation_dic' dictionary using the 'temp' dataframe.
        
    for country,abbr in countries.items():
        temp=pd.read_csv(f'../Data Sources/ENTSO-E/2018/Generation/{country}.csv',low_memory=False)
        # temp = temp.replace(['n/e',np.nan] ,0)
        temp = temp.replace(['n/e'], 0)
        temp = temp.drop(['Hydro Pumped Storage  - Actual Consumption [MW]'],axis=1)

        temp=hourly_data(temp)
        
        for column in temp.columns.values:
            if(temp[column]==0).all():
                temp=temp.drop(column,axis=1)

        fuels = [x[:-26] for x in temp.columns.values]
        temp.columns = fuels

        for fuel in fuels:
            generation_data[f'{abbr} - {fuel}'] = temp[fuel]

        generation_dic[abbr] = temp

    return(generation_data,generation_dic)


# 3. Preparing Cross-border Transmission Data

In [79]:
def cross_border(abbr_list):

    transmission_data = pd.DataFrame()
    cross_border_data = pd.DataFrame()
    cross_border_data[['Day','Month','Year','Time']]=data[['Day','Month','Year','Time']]

# In the following command we get the list of the paths of all files in the directory. 
# Then one by one we copy each csv to 'temp' dataframe and make all the 'n/e' values of the 'temp' 0. 
# Then we cretae two new column in the 'transmission_data' dataframe and name it as the two country codes the power transmission occurs. 
# We use string editing to get the two country codes from the file path. 
# For example, in the power transmission occur between Germany and Austria, we name the column as 'DE -> AT' and if the power transmission occur between Austria and Germany, we name the column as 'AT -> DE '.
# We use 'pd.to_numeric' function to convert the string values to numerical values if any numeric values have been recorded as string in the datasets. 
# Then we send the numeric converted column to 'hourly_data' function because cross border trasnmissions occur between Germany and a some countries have 35044 data points but We need to convert them to hourly values.
# Then we make a list of column heads of imports and exports associated with a given country_abbreviation 
# After this step we assume imports power transmission as a negative value and exports transmission as a positive value of a given country. 
# Therefore we multiply the 'imports' columns of the 'transmission_data' dataframe by -1 and add the 'exports' columns of the 'transmission_data' to get the net inbound/outbound in that country in that particular time step and save that in the 'cross_border_data' dataframe.
# Then we filter the transmission links between given two countries in which if both countries associated with the power transmission are included in our country_abbreviation list.
# For example power import(export) occurs from(to) a country other than the countries in the abbreviation_list (for ex: 'Cyprus','Turkey' etc.) are omitted. 
# Then we select only those filtered columns in the 'transmission_data' dataframe.

    csvs = glob.glob("../Data Sources/ENTSO-E/2018/Transmission/*.csv")

    for csv in csvs:
        temp = pd.read_csv(csv)
        # temp = temp.replace(['n/e', np.nan], 0)
        temp = temp.replace(['n/e'], 0)

        transmission_data[f'{csv[42:44]} - > {csv[45:47]}'] = hourly_data(pd.to_numeric(temp.iloc[:, 2]))
        transmission_data[f'{csv[45:47]} - > {csv[42:44]}'] = hourly_data(pd.to_numeric(temp.iloc[:, 1]))

    for abbr in abbr_list:
        imports = [x for x in transmission_data.columns.values if abbr in x[-2:]]
        exports = [x for x in transmission_data.columns.values if abbr in x[:2]]

        cross_border_data[f'{abbr}'] = transmission_data[exports].sum(axis=1) + (transmission_data[imports].sum(axis=1))*-1

    transmission_lines = list([x for x in transmission_data.columns.values if x[:2] in abbr_list and x[-2:] in abbr_list])
    transmission_data = transmission_data[transmission_lines]

    return cross_border_data, transmission_data


## 3.1 Calculating net imports/exports based on generation and load data 

In [80]:
def import_export_using_load_gen(load_data, generation_data, abbr_list):

    import_export_data = pd.DataFrame()
    import_export_data[['Day', 'Month', 'Year', 'Time']] = load_data[['Day', 'Month', 'Year', 'Time']]

    # In the following command we calculate net import/export in each time step of each country by subtracting '{country_code}' column of 'load_data' dataframe from '{country_code} - Total' of 'generation_data' dataframe and save the result in '{country_code} - [gen - load]' column of 'import_export_data' dataframe.
    # for example, import_export_data['DE - [gen - load]']=generation_data['DE - Total'] - load_data['DE'].
    # Then we create a new column in the 'import_export_data' dataframe called '{country_code} - import/export' and make that column 'Net Export' if '{country_code} - [gen - load]' is greater than 0 and make the '{country_code} - import/export' column 'Net Import' if '{country_code} - [gen - load]' is equal or lower than 0

    for abbr in abbr_list:
        import_export_data[f'{abbr}'] = generation_data.filter(like=abbr).sum(axis=1) - load_data[f'{abbr}']

    return(import_export_data)


## 3.2 Calculating net imports/exports based on cross-border transmission data 

In [81]:
def import_export_using_crossborder(crossborder_data, abbr_list):

    import_export_data = pd.DataFrame()
    import_export_data[['Day', 'Month', 'Year', 'Time']] = crossborder_data[['Day', 'Month', 'Year', 'Time']]

# In the following command we copy the '{country_code}' column of 'crossborder_data' dataframe to '{country_code} - [exp - imp]' column of 'import_export_data' dataframe.
# for example, import_export_data['DE - [exp - imp]'].
# Then we create a new column in the 'import_export_data' dataframe called '{country_code} - import/export' and make that column 'Net Export' if '{country_code} - [exp - imp]' is greater than 0 and make the '{country_code} - import/export' column 'Net Import' if '{country_code} - [exp - imp]' is equal or lower than 0

    for abbr in abbr_list:
        import_export_data[f'{abbr}'] = crossborder_data[f'{abbr}']

    return(import_export_data)


# 4. Internal sigma calculation

## 4.1 Alpha calculation

In [82]:
# def calculate_alpha(load_data,generation_data,transmission_data,abbr_list):

#     eph = 0.1
#     A=100
#     load_gen_data = {}
#     alpha = {}

#     # First we create a new dictionary called 'load_gen_data' and in that dictionary keys are country_abbreviations and as value of each key we add the combined demand column of each country and generation columns from all the sources in that country.

#     for abbr, df in generation_data.items():
#         load_gen_data[abbr] = pd.concat([df, load_data[abbr]],axis=1)

#     # In the following section we take each generation,load and transmission csvs and send them to the 'neareset_neighbours_mean' function.
#     # This function send each dataframe to 'omit_dst' function and removes the empty observations on March 28th and get the 'divider' value which is associated with the length of the dataframe.
#     # Then we replace 'n/e' values with 0
#     # Then we check the dataframe column by column.
#     # First we filter the columns in which the whole column is not 0 in the dataframe.
#     # Then we add the index and value of each value in the column to a list called 'column_data'
#     # Then we check value by value in the column untill a null value is found.
#     # If the value is a null value, we get all the values in that column which are the values associated with the same time step of the null value and the absolute difference of the indexes of that value and the null value to the 'temp_list'.
#     # For example if the null value occurus at 10:00 - 10:15 time step of some day, all the values associated with 10:00 - 10:15 time step throughout the year and and the absolute difference of the indexes of those value and the null value go to the 'temp_list'
#     # Then the 'temp_list' is sorted and we get all the indexes in the 'temp_list' except the index of the null value to a 2D numpy array 'X'.
#     # We get all the values in the 'temp_list' except the value of the null value to a 1D numpy array 'y'
#     # Then we divide 'X' and 'y' values in the ratio of 30% test and 70% train data.
#     # We create an array of degree values from 1 to 10.
#     # Then we iterate the 'degrees' one by one and create polynomial values of 'x_train' data called 'x_poly_train' based on the value of the degree
#     # Then we fit the polynomial linear regression function using 'x_poly_train' data and 'y_train' data.
#     # Then based on the polynomial function, using the 'x_poly_test' data we predict the values of the 'y_test' data
#     # Then based on the predicted values and 'y_test" data we calculate the Root Mean Square Error.
#     # Applying the last 4 steps for each degree value, we select the degree value which gives the Lowest Root Mean Square Error.
#     # Then we fit the polynomial linear regression function again using that degree which gives the Lowest Root Mean Square Error.
#     # Based on the polynimial function we get the predicted value of the null value.

#     for abbr, df in load_gen_data.items():
#         for column in df.columns.values:
#             column_data = [[index, value] for index, value in enumerate(df.loc[:, column])]
#             for index, value in column_data:
#                 if pd.isnull(value):
#                     temp_list = [[abs(index-item[0]), item[1]] for item in column_data if (index-item[0]) % 24 == 0]
#                     temp_list.sort()
#                     X = np.array([i[0] for i in temp_list[1:]]).reshape(len(temp_list)-1, 1)
#                     y = [i[1] for i in temp_list[1:]]
#                     y = [0 if pd.isna(x) else x for x in y]

#                     x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#                     degrees = np.arange(1, 11)
#                     min_rmse, min_deg = 1e10, 0

#                     for degree in degrees:

#                         # Preparing polynomial Train features based on x_train
#                         poly_features = PolynomialFeatures(degree=degree, include_bias=False)
#                         x_poly_train = poly_features.fit_transform(x_train)

#                         # Polynomial linear regression based on train data
#                         poly_reg = LinearRegression()
#                         poly_reg.fit(x_poly_train, y_train)

#                         # Predicting y values and getting root mean squared error based on predicted y values and y_test values
#                         x_poly_test = poly_features.fit_transform(x_test)
#                         poly_predict = poly_reg.predict(x_poly_test)
#                         poly_mse = mean_squared_error(y_test, poly_predict)
#                         poly_rmse = np.sqrt(poly_mse)

#                         # Selecting the best degree of the polynimial function based on lowest root mean squared error
#                         if min_rmse > poly_rmse:
#                             min_rmse = poly_rmse
#                             min_deg = degree

#                     # Fitting the regression function again based on the selected best degree above
#                     poly_features = PolynomialFeatures(degree=min_deg, include_bias=False)
#                     x_poly_train = poly_features.fit_transform(x_train)
#                     poly_reg = LinearRegression()
#                     poly_reg.fit(x_poly_train, y_train)

#                     df.loc[index, column] = poly_reg.predict(poly_features.fit_transform([[0]]))[0]
#             df[column] = df[column].apply(lambda x: eph if x < eph else x)
#             df[column] = df[column].apply(lambda x:  A/x)
#         alpha[abbr] = df

#     for column in transmission_data.columns.values:
#         column_data = [[index, value] for index, value in enumerate(transmission_data.loc[:, column])]
#         for index,value in column_data:
#             if pd.isnull(value):
#                 temp_list = [[abs(index-item[0]), item[1]] for item in column_data if (index-item[0]) % 24 == 0]
#                 temp_list.sort()
#                 X = np.array([i[0] for i in temp_list[1:]]).reshape(len(temp_list)-1, 1)
#                 y = [i[1] for i in temp_list[1:]]
#                 y = [0 if pd.isna(x) else x for x in y]

#                 x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#                 degrees = np.arange(1, 11)
#                 min_rmse, min_deg = 1e10, 0

#                 for degree in degrees:

#                     # Preparing polynomial Train features based on x_train
#                     poly_features = PolynomialFeatures(degree=degree, include_bias=False)
#                     x_poly_train = poly_features.fit_transform(x_train)

#                     # Polynomial linear regression based on train data
#                     poly_reg = LinearRegression()
#                     poly_reg.fit(x_poly_train, y_train)

#                     # Predicting y values and getting root mean squared error based on predicted y values and y_test values
#                     x_poly_test = poly_features.fit_transform(x_test)
#                     poly_predict = poly_reg.predict(x_poly_test)
#                     poly_mse = mean_squared_error(y_test, poly_predict)
#                     poly_rmse = np.sqrt(poly_mse)

#                     # Selecting the best degree of the polynimial function based on lowest root mean squared error
#                 if min_rmse > poly_rmse:
#                     min_rmse = poly_rmse
#                     min_deg = degree

#                 # Fitting the regression function again based on the selected best degree above
#                 poly_features = PolynomialFeatures(degree=min_deg, include_bias=False)
#                 x_poly_train = poly_features.fit_transform(x_train)
#                 poly_reg = LinearRegression()
#                 poly_reg.fit(x_poly_train, y_train)

#                 transmission_data.loc[index,column]= poly_reg.predict(poly_features.fit_transform([[0]]))[0]
#         transmission_data[column] = transmission_data[column].apply(lambda x: eph if x < eph else x)
#         transmission_data[column] = transmission_data[column].apply(lambda x:  A/x)
#     alpha["transmission_data"] = transmission_data

#     return(alpha)


## 4.2 Sigma calculation

In [83]:
def calculate_sigma(load_data,generation_data,transmission_data,abbr_list):

    eph = 0.1
    A=100
    load_gen_data = {}
    sigma = {}

    # First we create a new dictionary called 'load_gen_data' and in that dictionary keys are country_abbreviations and as value of each key we add the combined demand column of each country and generation columns from all the sources in that country.

    for abbr, df in generation_data.items():
        load_gen_data[abbr] = pd.concat([df, load_data[abbr]],axis=1)

    # Then we check the dataframe column by column.
    # Then we add the index and value of each value in the column to a list called 'column_data'
    # Then we check value by value in the column untill a null value is found.
    # If the value is a null value, we get all the values in that column which are the values associated with the same time step of the null value and the absolute difference of the indexes of that value and the null value to the 'temp_list'.
    # For example if the null value occurus at 10:00 - 10:15 time step of some day, all the values associated with 10:00 - 10:15 time step throughout the year and and the absolute difference of the indexes of those value and the null value go to the 'temp_list'
    # Then the 'temp_list' is sorted and the first 60 values are taken to the 'moving_average' list
    # Here in the 'moving_average' list we get the values of the nearest 60 days of the same time step in which the null value has occured.
    # Then we replace any null values in the 'moving_average' list with 0
    # Then we get the mean value of the 'moving_average' list and assign that value to the original null value in the dataframe

    for abbr, df in load_gen_data.items():
        for column in df.columns.values:
            column_data = [[index, value] for index, value in enumerate(df.loc[:, column])]
            for index, value in column_data:
                if pd.isnull(value):
                    temp_list = [[abs(index-item[0]), item[1]] for item in column_data if (index-item[0]) % 24 == 0]
                    temp_list.sort()
                    moving_average = [x[1] for x in temp_list[1:11]]
                    moving_average = [0 if pd.isna(x) else x for x in moving_average]
                    df.loc[index, column] = np.mean(np.array(moving_average))
            df[column] = df[column].apply(lambda x: eph if x < eph else x)
            df[column] = df[column].apply(lambda x:  A/x)
        sigma[abbr] = df

    for column in transmission_data.columns.values:
        column_data = [[index, value] for index, value in enumerate(transmission_data.loc[:, column])]
        for index,value in column_data:
            if pd.isnull(value):
                temp_list = [[abs(index-item[0]), item[1]] for item in column_data if (index-item[0]) % 24 == 0]
                temp_list.sort()
                moving_average = [x[1] for x in temp_list[1:11]]
                moving_average = [0 if pd.isna(x) else x for x in moving_average]
                transmission_data.loc[index,column]= np.mean(np.array(moving_average))
        transmission_data[column] = transmission_data[column].apply(lambda x: eph if x < eph else x)
        transmission_data[column] = transmission_data[column].apply(lambda x:  A/x)
    sigma["transmission_data"] = transmission_data

    return(sigma)


## 4.3 Internal data consolidation

In [86]:
def data_consolidation(method,generation_dic_copy, load_dic_copy, transmission_data_copy, intermediary_var, unit_var):
    consolidated_gen_data = {}
    consolidated_load_data = {}
    consolidated_transmission_data = {}

    # In the following command, we fill each generation by source value in each timestep in each country with ('intermediary_var["generation"]' + 'original_value' * 'unit_var['generation]') value of that value
    # Similarly we fill each load value in each timestep in each country with ('intermediary_var['load'] + 'original_value' * 'unit_var['load']') value of that value
    # Then we save the consolidated generation and load values in seperate csv files.
    # We follow similar steps to obtain consolidated transmission values and save the result in a seperate csv file. 

    for abbr, df in generation_dic_copy.items():
        for column in df.columns:
            df[column] = intermediary_var["generation"][abbr][column] + \
                df[column] * unit_var["generation"][abbr][column]
        consolidated_gen_data[abbr] = df
        consolidated_load_data[abbr] = intermediary_var["load"][abbr]['demand'] + \
            load_dic_copy[abbr]['demand'] * unit_var["load"][abbr]['demand']
        consolidated_gen_data[abbr].to_csv(f"../Data Sources/output/{method}/Generation/{abbr}.csv")
        consolidated_load_data[abbr].to_csv(f"../Data Sources/output/{method}/Load/{abbr}.csv")

    for column in transmission_data_copy.columns:
        transmission_data_copy[column] = intermediary_var["transmission"][column] + \
            transmission_data_copy[column] * unit_var["transmission"][column]
    consolidated_transmission_data = transmission_data_copy
    consolidated_transmission_data.to_csv(f'../Data Sources/output/{method}/Transmission/all_transmissions.csv')

In [85]:
# sigma(load(countries)[1], generation(countries)[1], cross_border(abbr_list)[1],abbr_list)